# webGL a jupyter-ben

Ehhez kell a [pythreejs](https://github.com/jovyan/pythreejs) csomag..

In [1]:
%pylab inline
#% config InlineBackend.figure_format = 'svg'
from pythreejs import *
import numpy as np
from IPython.display import display, clear_output, HTML
from ipywidgets import interact, interactive
from ipywidgets import HBox
from io import BytesIO 
from ipywidgets import HTML, Text
from traitlets import link, dlink

import base64 



Populating the interactive namespace from numpy and matplotlib


In [2]:
def eqimg(eqstr):
    sio2=BytesIO();
    figsize(2,2);
    fig=figure();
    ax=fig.add_subplot(111);
    ax.text(0,0.3,eqstr,fontsize=60);
    ax.axis('off');
    fig.savefig(sio2, format="png",transparent=True)
    figstr='data:image/png;base64,'+(base64.encodebytes(sio2.getvalue()).decode());
    return figstr

def threeaxes(l,arrl):
    linesgeom = PlainGeometry(vertices=[[-l,0,0],[l,0,0],
                                        [0,-l,0],[0,l,0],
                                        [0,0,-l],[0,0,l],
                                        [l-arrl,-arrl,0],[l,0,0],
                                        [l-arrl,arrl,0],[l,0,0],
                                        [-arrl,l-arrl,0],[0,l,0],
                                        [arrl,l-arrl,0],[0,l,0],
                                       [-arrl,0,l-arrl],[0,0,l],
                                        [arrl,0,l-arrl],[0,0,l],],
                          colors = ['black' for i in range(18)])
    return Line(geometry=linesgeom, 
             material=LineBasicMaterial( linewidth=1, vertexColors='VertexColors'), 
             type='LinePieces')

In [3]:
%%capture
axd=Sprite(position=[2.6,0.3,-0.25],scaleToTexture=True,scale=[1.0,1.0,1.0],
           material=SpriteMaterial(map=ImageTexture(imageuri=eqimg(r'$d_x$'))))
ayd=Sprite(position=[0.3,2.6,-0.25],scaleToTexture=True,scale=[1.0,1.0,1.0],
           material=SpriteMaterial(map=ImageTexture(imageuri=eqimg(r'$d_y$'))))
azd=Sprite(position=[-0.6,0.3,2.6],scaleToTexture=True,scale=[1.0,1.0,1.0],
           material=SpriteMaterial(map=ImageTexture(imageuri=eqimg(r'$d_z$'))))


orig = Mesh(geometry=SphereGeometry(radius=0.1),material=BasicMaterial(color = 0x000000));

In [4]:
def bhzp2_js(ub,ue,vb,ve,Delta):
     return """
        bhzp2 = function (u, v) {
            var x = Math.sin((u*%.3f+%.3f) * 2 * Math.PI);
            var y = Math.sin((v*%.3f+%.3f) * 2 * Math.PI);
            var z = (Math.cos((u*%.3f+%.3f) * 2 * Math.PI) + Math.cos((v*%.3f+%.3f) * 2 * Math.PI)) +%.3f ;
            return new THREE.Vector3(x, y, z);
          }
        """%tuple([ue,ub,ve,vb,ue,ub,ve,vb,Delta])    

    
def look_at_bzhp2(ub,ue,vb,ve,Delta):    
    surf_g = ParametricGeometry(func=bhzp2_js(ub,ue,vb,ve,Delta),slices=15,stacks=15);
    surf = Mesh(geometry=surf_g,material=LambertMaterial(color = 0xff0000,side ='FrontSide'));
    surf2 = Mesh(geometry=surf_g,material=LambertMaterial(color = 0x00ffff,side ='BackSide'));  
    return  Scene(children=[surf,surf2,threeaxes(2.6,0.15),orig,axd,ayd,azd,AmbientLight(color=0x101010)]);
        

c1 = PerspectiveCamera(position=[4,4,4], up=[0,0,1],
                      aspect =1.0,children=[DirectionalLight(position=[-3,-3,-3], color=[255,255,255])]);
rnd = Renderer(scene=look_at_bzhp2(0,1.0,-0.5,0.5,-1.0),
               camera=c1,
               controls=[OrbitControls(controlling=c1)],
               background='white',
               width = 500,height = 500)


In [5]:
%%capture
axk=Sprite(position=[1.5*pi,0.3,-0.5],scaleToTexture=True,scale=[1.6,1.6,1.6],
           material=SpriteMaterial(map=ImageTexture(imageuri=eqimg(r'$k_x$'))))
ayk=Sprite(position=[0.3,1.5*pi,-0.5],scaleToTexture=True,scale=[1.6,1.6,1.6],
           material=SpriteMaterial(map=ImageTexture(imageuri=eqimg(r'$k_y$'))))
azE=Sprite(position=[-0.5,1.3,1.5*pi],scaleToTexture=True,scale=[1.6,1.6,1.6],
           material=SpriteMaterial(map=ImageTexture(imageuri=eqimg(r'$E(\vec{k})$'))))


nx,ny=(25,25)
xmax=pi
x = np.linspace(-xmax,xmax,nx)
y = np.linspace(-xmax,xmax,ny)
xx, yy = np.meshgrid(x,y)
z = sqrt(sin(xx)**2+sin(yy)**2+(cos(xx)+cos(yy)+(-2.0))**2)

surf_ge = SurfaceGeometry(z=list(z[::-1].flat),
                          width=2*xmax,height=2*xmax,width_segments=nx-1,height_segments=ny-1)
surf_gh = SurfaceGeometry(z=list((-z[::-1]).flat),
                          width=2*xmax,height=2*xmax,width_segments=nx-1,height_segments=ny-1)

surfe = Mesh(geometry=surf_ge, material=PhongMaterial(map=height_texture(z[::-1], 'jet')))
surfh = Mesh(geometry=surf_gh, material=PhongMaterial(map=height_texture(-z[::-1], 'jet_r')))

scene = Scene(children=[surfe,
                        surfh,
                        threeaxes(1.5*pi,0.3), 
                        axk,ayk,azE,
                        AmbientLight(color=0x101010)])
c2 = PerspectiveCamera(position=[6,13,4], up=[0,0,1],
                      aspect =1.0,
                      children=[DirectionalLight(color='white', position=[3,5,1], intensity=1)])

rnd2 = Renderer(camera=c2, scene = scene, controls=[OrbitControls(controlling=c2)],
               background='white',
               width = 500,height = 500)



#### Spectrum and $d$ space configuration of the QWZ model

\begin{eqnarray*}
H=\mathbf{d}(\mathbf{k})\cdot\sigma\\
E_\pm(\mathbf{k})=\pm|\mathbf{d}(\mathbf{k})|
\end{eqnarray*}



In [6]:
@interact(u=(-3,3,0.025),p=(0.025,1,0.025))
def play(u=-1,p=0.025):
    rnd.scene.children[0].geometry=ParametricGeometry(func=bhzp2_js(0,1.0,-0.5,p,u),slices=25,stacks=25)
    rnd.scene.children[1].geometry=ParametricGeometry(func=bhzp2_js(0,1.0,-0.5,p,u),slices=25,stacks=25)
    zz=sqrt(sin(xx)**2+sin(yy)**2+(cos(xx)+cos(yy)+(u))**2);
    rnd2.scene.children[0].geometry.z=list((zz)[::-1].flat)
    #rnd2.scene.children[0].material=LambertMaterial(map=height_texture(zz[::-1], 'jet'));
    rnd2.scene.children[0].material=LambertMaterial(color='red');
    rnd2.scene.children[1].geometry.z=list((-zz)[::-1].flat)
    #rnd2.scene.children[1].material=LambertMaterial(map=height_texture(-zz[::-1], 'jet_r'));
    rnd2.scene.children[1].material=LambertMaterial(color='blue');
display(HBox([rnd, rnd2]))